In [1]:
%matplotlib inline
import numpy as np                  
import matplotlib.pyplot as plt     
import random   
import math
from tqdm import tqdm


from keras.datasets import mnist    
from keras.models import Sequential 

#from keras.layers.core import Dense, Dropout, Activation 
#from keras.utils import np_utils  

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

import matplotlib.image as mpimg
import scipy.ndimage as ndi

from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import keras

from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from numba import jit
pi = math.pi

import random
#from cv2 import cv2
import cv2
import scipy.stats
import seaborn as sns

from nexusformat.nexus import *
from matplotlib.patches import Circle



# Parameters

In [2]:
a_bulk = 3.905 #SrTiO3 is the bulk
a = a_bulk

c_bulk = 3.905 
c_surf = 3.79 #LaAlO3

#Nc = 1000
N_bulk = 8 + 1 #the plus 1 is the dummyy layer
N_surface = 4
#total 12 layer film (vary)

#rn describes the bulk atomic xyz positions
r1 = [0, 0, 0] #Sr
r2 = [0.5, 0.5, 0.5] #Ti
r3 = [0.5, 0.5, 0] #O
r4 = [0, 0.5, 0.5] #O
r5 = [0.5, 0, 0.5] #O

#hard setting the base atom position of the variable bulk and surface
rb1 = [[0, 0, 0]] * N_bulk
s1 = [[0, 0, 0]] * N_surface

#variation of atomic positions
#v = 0.05

In [3]:
#creating q mesh
a_SrTiO3 = 3.905
a = a_SrTiO3
dqz = 0.025 * (2*pi/a_SrTiO3)
z_size = 6 * (2*pi/a_SrTiO3)



rod_dimension = 4
dqxy = (2*pi)/a_SrTiO3
xy_size = (rod_dimension-1)*dqxy


x = np.linspace(0, xy_size, rod_dimension)
y = x
z = np.linspace(dqz, z_size, round(z_size/dqz))


[qx, qy, qz] = np.meshgrid(x, y, z)

z_dim = qx.shape[2]
xy_dim = qx.shape[0] * qx.shape[1]


print("XY Dimension: ", qx.shape[0]* qx.shape[1] )
print("Z Dimension: ", qx.shape[2] )
print("Total Points: ", qx.shape[0]* qx.shape[1] *qx.shape[2] )

XY Dimension:  16
Z Dimension:  240
Total Points:  3840


In [4]:
q_mag_squared_term = (qx**2 + qy**2 + qz**2)/((4*pi)**2)
def get_fq(a_arr, b_arr, c_val):
    fq = np.zeros(qx.shape)
    for i in range(4):
        fq += a_arr[i] * np.exp(-b_arr[i] * q_mag_squared_term ) 
    fq += c_val
    return fq

In [5]:
# Define the file path
file_path = 'LSMO_newdat2.XYZ'  # Replace with the actual file path

with open(file_path, 'r') as xyz_file:
    lines = xyz_file.readlines()[2:] # Skipping the first two lines

In [6]:
label = []
x_cobra = []
y_cobra = []
z_cobra = []
DW_factor = []
p_cobra = []

for i in range(len(lines)):
    row = lines[i]
    arr = row.split()
    label.append((arr[0]))
    x_cobra.append(float(arr[1]))
    y_cobra.append(float(arr[2]))
    z_cobra.append(float(arr[3]))
    DW_factor.append(float(arr[4]))
    p_cobra.append(float(arr[5]))
    
    
U_sr = DW_factor[0]
U_la = DW_factor[1]
U_o1 = DW_factor[2]
U_ti = DW_factor[3]
U_mn = DW_factor[4]
U_o3 = DW_factor[5]
U_o2 = DW_factor[6]

In [7]:
#Definining DW Factors
q_squared = (qx**2 + qy**2 + qz**2)


DW_sr = np.exp( (-1/2)*q_squared* U_sr  )
DW_la = np.exp( (-1/2)*q_squared* U_la  )
DW_ti = np.exp( (-1/2)*q_squared* U_ti  )
DW_mn = np.exp( (-1/2)*q_squared* U_mn  )
DW_o1 = np.exp( (-1/2)*q_squared* U_o1  )
DW_o2 = np.exp( (-1/2)*q_squared* U_o2  )
DW_o3 = np.exp( (-1/2)*q_squared* U_o3  )

In [8]:
#change to whatever bulk and surface you're using 
a_Sr = [18.0874, 8.1373, 2.5654, -34.193]
b_Sr = [1.4907, 12.6963, 24.5651, -0.0138]
c_Sr = 41.4025

a_Ti = [19.5114, 8.23473, 2.01341, 1.5208]
b_Ti = [0.178847, 6.67018, -0.29263, 12.9464]
c_Ti = -13.28

a_O = [3.0485, 2.2868, 1.5463, 0.867]
b_O = [13.2771, 5.7011, 0.3239, 32.9089]
c_O = 0.2508

a_La = [20.2489, 19.3763, 11.6323, 0.336048]
b_La = [2.9207, 0.250698, 17.8211, 54.9453]
c_La = 2.4086

a_Al = [4.17448, 3.3876, 1.20296, 0.528137]
b_Al = [1.93816, 4.14553, 0.228753, 8.28524]
c_Al = 0.706786

fq_Sr = get_fq(a_Sr, b_Sr, c_Sr)
fq_Ti = get_fq(a_Ti, b_Ti, c_Ti)
fq_O =  get_fq(a_O, b_O, c_O)
fq_La = get_fq(a_La, b_La, c_La)
fq_Al =  get_fq(a_Al, b_Al, c_Al)

#fq for the bulk and surface
#fq_b = [fq_Sr, fq_Ti, fq_O, fq_O, fq_O]
#fq_s = [fq_La, fq_Al, fq_O, fq_O, fq_O]

fq_b = [fq_Sr*DW_sr, fq_Ti*DW_ti, fq_O*DW_o1, fq_O*DW_o2, fq_O*DW_o3]
fq_s = [fq_La*DW_la, fq_Al*DW_mn, fq_O*DW_o1, fq_O*DW_o2, fq_O*DW_o3]


In [9]:
#an array that is zero at the bragg peaks and one everywhere else
#inverse is the opposite
bragg_peak_arr = np.ones(qx.shape)
bragg_peak_arr_inverse = np.zeros(qx.shape)

for i in range(len(bragg_peak_arr)):
    for j in range(len(bragg_peak_arr[0])):
        for k in range(len(bragg_peak_arr[0][0])):
            if np.allclose(z[k]/(2*pi/a_bulk) % 1,  0):
                bragg_peak_arr[i][j][k] = 0
                bragg_peak_arr_inverse[i][j][k] = 1
                
                
bragg_peak_arr_flat = bragg_peak_arr.reshape(xy_dim, z_dim)

# Simulation Methods

In [10]:
#Z scattering
#z_scat_bulk = np.zeros(qx.shape)
#z_scat_bulk = 0.5/np.sin(c_bulk*qz/2)

#simulate general ctr scattering 
def createCTR_allr(r1, r2, r3, r4, r5, rb1, rb2, rb3, rb4, rb5, s1, s2, s3, s4, s5, cn_bulk, cn_surf, mix):
    
   
    z_scat_bulk = np.zeros(qx.shape,dtype=np.complex_)

    nom = 1
    denom = (np.exp(1j * qz * c_bulk)-1)
    z_scat_bulk = (nom/denom)
        
        
    F_q_bulk = np.zeros(qx.shape)

    F_q_bulk = F_q_bulk + fq_b[0]*np.exp( 1j * a_bulk*(qx*r1[0] + qy*r1[1] + qz*r1[2])) #Sr
    F_q_bulk = F_q_bulk + fq_b[1]*np.exp( 1j * a_bulk*(qx*r2[0] + qy*r2[1] + qz*r2[2])) #Ti
    F_q_bulk = F_q_bulk + fq_b[2]*np.exp( 1j * a_bulk*(qx*r3[0] + qy*r3[1] + qz*r3[2])) #O
    F_q_bulk = F_q_bulk + fq_b[3]*np.exp( 1j * a_bulk*(qx*r4[0] + qy*r4[1] + qz*r4[2])) #O
    F_q_bulk = F_q_bulk + fq_b[4]*np.exp( 1j * a_bulk*(qx*r5[0] + qy*r5[1] + qz*r5[2])) #O
    
    bulk = z_scat_bulk*F_q_bulk 
    
    #-------
    
    c_sum = 0
    bulk_variable = np.zeros(qx.shape)
    for i in range(0, N_bulk):
            
        F_q_bulk_var = np.zeros(qx.shape)
        f_mixed = (1-mix[i])*fq_s[0] + mix[i]*fq_b[0]
        F_q_bulk_var = F_q_bulk_var + f_mixed*np.exp( 1j * a_bulk*(qx*rb1[i][0] + qy*rb1[i][1]) + 1j*cn_bulk[i]*(qz*rb1[i][2])) #Sr

        F_q_bulk_var = F_q_bulk_var + fq_b[1]*np.exp( 1j * a_bulk*(qx*rb2[i][0] + qy*rb2[i][1]) +  1j*cn_bulk[i]*(qz*rb2[i][2])) #Ti
        F_q_bulk_var = F_q_bulk_var + fq_b[2]*np.exp( 1j*   a_bulk*(qx*rb3[i][0] + qy*rb3[i][1]) + 1j*cn_bulk[i]*(qz*rb3[i][2])) #O
        F_q_bulk_var = F_q_bulk_var + fq_b[3]*np.exp( 1j *  a_bulk*(qx*rb4[i][0] + qy*rb4[i][1]) + 1j*cn_bulk[i]*(qz*rb4[i][2])) #O
        F_q_bulk_var = F_q_bulk_var + fq_b[4]*np.exp( 1j *  a_bulk*(qx*rb5[i][0] + qy*rb5[i][1]) + 1j*cn_bulk[i]*(qz*rb5[i][2])) #O

        bulk_variable = bulk_variable + F_q_bulk_var*np.exp(1j* qz * c_sum)
        c_sum += cn_bulk[i]
    #-----
    
    
    
    surface = np.zeros(qx.shape)
    for i in range(N_bulk, N_bulk + N_surface):
        
        #if there's mixing
        F_q_surf = np.zeros(qx.shape)
        f_mixed = (1-mix[i])*fq_s[0] + mix[i]*fq_b[0]
        F_q_surf = F_q_surf + f_mixed*np.exp( 1j * a_bulk*(qx*s1[i - N_bulk][0] + qy*s1[i - N_bulk][1]) + 1j*cn_surf[i - N_bulk]*(qz*s1[i - N_bulk][2])) #La
        F_q_surf = F_q_surf + fq_s[1]*np.exp( 1j * a_bulk*(qx*s2[i - N_bulk][0] + qy*s2[i - N_bulk][1]) + 1j*cn_surf[i - N_bulk]*(qz*s2[i - N_bulk][2])) #Al
        F_q_surf = F_q_surf + fq_s[2]*np.exp( 1j*   a_bulk*(qx*s3[i - N_bulk][0] + qy*s3[i - N_bulk][1]) + 1j*cn_surf[i - N_bulk]*(qz*s3[i - N_bulk][2])) #O
        F_q_surf = F_q_surf + fq_s[3]*np.exp( 1j *  a_bulk*(qx*s4[i - N_bulk][0] + qy*s4[i - N_bulk][1]) + 1j*cn_surf[i - N_bulk]*(qz*s4[i - N_bulk][2])) #O
        F_q_surf = F_q_surf + fq_s[4]*np.exp( 1j *  a_bulk*(qx*s5[i - N_bulk][0] + qy*s5[i - N_bulk][1]) + 1j*cn_surf[i - N_bulk]*(qz*s5[i - N_bulk][2])) #O

        surface = surface + F_q_surf*np.exp(1j * qz * c_sum)
        c_sum += cn_surf[i - N_bulk]
    

    e_field = (bulk + bulk_variable + surface)
    ctr = np.abs(bulk + bulk_variable + surface) ** 2
    phase = np.angle(e_field)
        
    
    return ctr, e_field, phase




def createCTR_reshaped_log_allr(r1, r2, r3, r4, r5, rb1, rb2, rb3, rb4, rb5, s1, s2, s3, s4, s5, cn_bulk, cn_surf, mix, beta):
    
   
    
    epsilon = 0.00
    
    ctr, e_field, phase = createCTR_allr(r1, r2, r3, r4, r5, 
                    rb1, rb2, rb3, rb4, rb5, 
                    s1, s2, s3, s4, s5, cn_bulk, cn_surf, mix)
    
    ctr_reshaped = ctr.reshape(xy_dim, z_dim)
    #Roughness parameter
    for rod in range(len(ctr_reshaped)):
        for l in range(len(ctr_reshaped[0])):
            scaling_factor = (1-beta**2)/(1 + beta**2 - 2*beta*np.cos(z[l] * c_surf))
            ctr_reshaped[rod][l] *=  scaling_factor
            
    return np.log(ctr_reshaped + epsilon), e_field, phase


def generatePs():
    generated = False
    betaVal = 0.75
    while (not generated):
        p1 = np.random.beta(betaVal, betaVal)
        p2 = np.random.beta(betaVal, betaVal)
        p3 = np.random.beta(betaVal, betaVal)
        p4 = np.random.beta(betaVal, betaVal)
        tot = (p1 + p2 + p3 + p4)/2
        p1/=tot
        p2/=tot
        p3/=tot
        p4/=tot
        if (p1 > 1 or p2 > 1 or p3 > 1 or p4 > 1):
            continue
        generated = True
        
    return p1, p2, p3, p4

def generateCs(c, n, v):
    c_arr = [c] * n
    for i in range(n):
        c_arr[i] = random.uniform(c-v, c+v)
        
    return c_arr
        
def removeBraggPeaks(CTR, thresh):
    peak_locations = []
    for i in range(len(CTR[0])):
        if CTR[0][i] > thresh:
            peak_locations.append(i)

    return np.delete(CTR, peak_locations, axis = 1)

def generatePositions(positions, v, n):
    defaultPos = np.array([positions] * n)
   
    for i in range(n):
        #2 because we change the z position
        defaultPos[i][2] = random.uniform(positions[2]-v, positions[2]+v)
    return defaultPos

#So the two equatorial oxygens have the same z position
def generateTwoPositions(positions1, positions2, v, n):
    defaultPos1 = np.array([positions1] * n)
    defaultPos2 = np.array([positions2] * n)
   
    for i in range(n):
        #2 because we change the z position
        randZ = random.uniform(positions1[2]-v, positions1[2]+v)
        defaultPos1[i][2] = randZ
        defaultPos2[i][2] = randZ
        
    return defaultPos1, defaultPos2


#MODIFYING THIS METHOD
def getAtomDiffs(cn_bulk, cn_surf, rb, s, zpos, N_bulk, N_surface):
    
    c_B_arr = [0] * N_bulk
    c_S_arr = [0] * N_surface
    
    
    for i in range(len(c_B_arr)):
        if i == 0:
            c_B_arr[i] = c_bulk + cn_bulk[i]*rb[i][2] - c_bulk*zpos
            
        else:
            c_B_arr[i] = cn_bulk[i-1] + cn_bulk[i]*rb[i][2] - cn_bulk[i-1]*rb[i-1][2]
       
    for i in range(len(c_S_arr)):
        if i == 0:
            c_S_arr[i] = cn_bulk[-1] + cn_surf[i]*s[i][2] - cn_bulk[-1]*rb[-1][2]
        else:
            c_S_arr[i] = cn_surf[i-1] + cn_surf[i]*s[i][2] - cn_surf[i-1]*s[i-1][2]
            
    return c_B_arr + c_S_arr
    
    
def removeBraggPeaks_messy(CTR, bragg_width, thresh):
    #only works if we use a nice number for z!
    peak_locations = []
    totLength = len(CTR[0])
    totRemoved = 0
    for i in range(len(CTR[0])):
        if CTR[0][i] > thresh:
            randArea = np.random.randint(bragg_width)
            
            if i + randArea < totLength:
                peak_locations = np.concatenate([peak_locations, (np.linspace(i-randArea, i + randArea, 2*randArea+1))])
            else:
                peak_locations = np.append(peak_locations, i)
    padding = np.zeros((len(CTR), len(set(peak_locations))))
        
    deleted = np.delete(CTR, peak_locations.astype(int), axis = 1)
    final_padded = np.concatenate([deleted, padding], axis = 1)
    
    return final_padded

#### S(q) method

In [11]:
def get_Sq(r1, r2, r3, r4, r5):
    
    z_scat_bulk = np.zeros(qx.shape,dtype=np.complex_)

    nom = 1
    denom = (np.exp(1j * qz * c_bulk)-1)
    z_scat_bulk = (nom/denom)
        
        
    F_q_bulk = np.zeros(qx.shape)

    F_q_bulk = F_q_bulk + fq_b[0]*np.exp( 1j * a_bulk*(qx*r1[0] + qy*r1[1] + qz*r1[2])) #Sr
    F_q_bulk = F_q_bulk + fq_b[1]*np.exp( 1j * a_bulk*(qx*r2[0] + qy*r2[1] + qz*r2[2])) #Ti
    F_q_bulk = F_q_bulk + fq_b[2]*np.exp( 1j * a_bulk*(qx*r3[0] + qy*r3[1] + qz*r3[2])) #O
    F_q_bulk = F_q_bulk + fq_b[3]*np.exp( 1j * a_bulk*(qx*r4[0] + qy*r4[1] + qz*r4[2])) #O
    F_q_bulk = F_q_bulk + fq_b[4]*np.exp( 1j * a_bulk*(qx*r5[0] + qy*r5[1] + qz*r5[2])) #O
    
    S_q = z_scat_bulk*F_q_bulk 
    
    return S_q

In [12]:
S_q = get_Sq(r1, r2, r3, r4, r5)

In [13]:
np.mean(S_q)

(-8.515494007993489-9648209089619.713j)

In [14]:
test = np.array([
    [0, 1, 2],
    [3, 4, 5]
])

test2 = np.concatenate(([[9, 9, 9]], test), axis = 0)
print(type(test2))

<class 'numpy.ndarray'>


In [15]:
a_bulk

3.905

# Simulation

In [16]:
X_data = []
y_data_real = []
y_data_imag = []

all_cns = []
all_o1s = []
all_o2s = []
all_o3s = []
all_Tis = []
all_ps = []
train_size = 100000
dataset_size = train_size + 1000 #The extra 1000 is for testing
v_r = 0.05
v_cn = 0.1

for i in tqdm(range(dataset_size)):
    
    #beta = random.uniform(0, 0.5) #random roughness between 0 and 0.5
    beta = 0 #no roughness
    
    p1, p2, p3, p4 = generatePs()
    mix = [1]*(N_bulk-2) + [p1, p2, p3, p4] + [0]*(N_surface-2)
    
    rb2 = generatePositions([0.5, 0.5, 0.5], v_r, N_bulk) #Ti
    rb3 = generatePositions([0.5, 0.5, 0], v_r, N_bulk) #O1
    rb4, rb5 = generateTwoPositions([0, 0.5, 0.5], [0.5, 0, 0.5], v_r, N_bulk ) #O2, O3
    
    
    s2 = generatePositions([0.5, 0.5, 0.5], v_r, N_surface)
    s3 = generatePositions([0.5, 0.5, 0], v_r, N_surface)
    s4, s5 = generateTwoPositions([0, 0.5, 0.5], [0.5, 0, 0.5], v_r, N_surface )
  
    cn_bulk = generateCs(c_bulk, N_bulk, v_cn)
    cn_surf = generateCs(c_surf, N_surface, v_cn)
    
    
    #Since the first layer is a dummy layer, set it back to default positions
    rb2[0] = [0.5, 0.5, 0.5]
    rb3[0] = [0.5, 0.5, 0]
    rb4[0] = [0, 0.5, 0.5]
    rb5[0] = [0.5, 0, 0.5]
    cn_bulk[0] = a_bulk
    
    
    diffs2 = getAtomDiffs(cn_bulk, cn_surf, rb2, s2, r2[2], N_bulk, N_surface)
    diffs3 = getAtomDiffs(cn_bulk, cn_surf, rb3, s3, r3[2], N_bulk, N_surface)
    diffs4 = getAtomDiffs(cn_bulk, cn_surf, rb4, s4, r4[2], N_bulk, N_surface)
    diffs5 = getAtomDiffs(cn_bulk, cn_surf, rb5, s5, r5[2], N_bulk, N_surface)
    
    all_cns.append(cn_bulk + cn_surf)
    all_o1s.append(diffs5)
    all_o2s.append(diffs4)
    all_o3s.append(diffs3)
    all_Tis.append(diffs2)
    all_ps.append([p1, p2, p3, p4])
    
    
    CTR, e_field, phase = createCTR_reshaped_log_allr(r1, r2, r3, r4, r5, rb1, rb2, rb3, rb4, rb5, s1, s2, s3, s4, s5, cn_bulk, cn_surf, mix, beta)
   
    #Removing the bragg peaks from the input I(q)
    X_data.append(bragg_peak_arr_flat*CTR)
    
    U_q = (e_field - S_q).reshape(xy_dim * z_dim)
    
    y_data_real.append(np.real(U_q))
    y_data_imag.append(np.imag(U_q))
    
print("0")

X_data = np.array(X_data)
y_data_real = np.array(y_data_real)
y_data_imag = np.array(y_data_imag)



means_real = []
std_real = []
means_imag = []
std_imag = []

print("1")
for i in range(len(y_data_real[0])):
    means_real.append(np.mean(y_data_real[:, i]))
    std_real.append(np.std(y_data_real[:, i]))
    
    means_imag.append(np.mean(y_data_imag[:, i]))
    std_imag.append(np.std(y_data_imag[:, i]))

    
    
print("2")
    
for i in tqdm(range(len(y_data_real))):
    for j in range(len(y_data_real[0])):
        y_data_real[i][j] = (y_data_real[i][j] - means_real[j])/std_real[j]
        
        y_data_imag[i][j] = (y_data_imag[i][j] - means_imag[j])/std_imag[j]
        
print("Done")


100%|████████████████████████████████████████████████████████| 101000/101000 [12:40<00:00, 132.87it/s]


0
1
2


100%|████████████████████████████████████████████████████████| 101000/101000 [03:35<00:00, 468.81it/s]

Done


In [17]:
y_data_real.shape

(101000, 3840)

In [18]:
X_data[0].shape

(16, 240)

In [19]:
test_size = 1000

In [20]:
x_new, y_new = X_data[0].shape

#train_size = int(0.8 * len(X_data))
x_train_noReshape = X_data[0:train_size]
y_train_real = y_data_real[0:train_size]
y_train_imag = y_data_imag[0:train_size]


x_test_noReshape = X_data[train_size:]
y_test_real = y_data_real[train_size:]
y_test_imag = y_data_imag[train_size:]


x_train = x_train_noReshape.reshape(train_size, x_new * y_new)
x_test = x_test_noReshape.reshape(test_size, x_new * y_new)

mean_ctr_val = np.mean(X_data)
std_ctr_val = np.std(X_data)
x_train = (x_train - mean_ctr_val)/std_ctr_val
x_test = (x_test - mean_ctr_val)/std_ctr_val

x_train = np.array(x_train).reshape(len(x_train), x_new * y_new, 1)
x_test = np.array(x_test).reshape(len(x_test), x_new * y_new, 1)
y_train_real = np.array(y_train_real).reshape(len(x_train), x_new * y_new, 1)
y_test_real = np.array(y_test_real).reshape(len(x_test), x_new * y_new, 1)

y_train_imag = np.array(y_train_imag).reshape(len(x_train), x_new * y_new, 1)
y_test_imag = np.array(y_test_imag).reshape(len(x_test), x_new * y_new, 1)

x_train = np.array(x_train)
x_test = np.array(x_test)

print(x_train.shape)
print(y_train_real.shape)
print("Mean Val: ", mean_ctr_val)
print("STD Val: ", std_ctr_val)

(100000, 3840, 1)
(100000, 3840, 1)
Mean Val:  6.37676816343382
STD Val:  1.9829749141582458


In [21]:
#training data
np.save("x_train_12L_100k.npy", x_train)
np.save("x_test_12L_100k.npy", x_test)

np.save("y_train_real_12L_100k.npy", y_train_real)
np.save("y_test_real_12L_100k.npy", y_test_real)

np.save("y_train_imag_12L_100k.npy", y_train_imag)
np.save("y_test_imag_12L_100k.npy", y_test_imag)


#mean, std arrays for transformation
np.save("means_real_12L_100k.npy", means_real)
np.save("std_real_12L_100k.npy", std_real)

np.save("means_imag_12L_100k.npy", means_imag)
np.save("std_imag_12L_100k.npy", std_imag)

#atomic positions
np.save("all_cns_12L_100k.npy", all_cns)
np.save("all_o1s_12L_100k.npy", all_o1s)
np.save("all_o2s_12L_100k.npy", all_o2s)
np.save("all_o3s_12L_100k.npy", all_o3s)
np.save("all_Tis_12L_100k.npy", all_Tis)
np.save("all_ps_12L_100k.npy", all_ps)
